# Lab 2: Instruct-Validate-Repair

Instruct-Validate-Repair is a design pattern for building robust automation using LLMs. The idea is simple:

1. Instruct the model to perform a task and specify requirements on the output of the task.
2. Validate that these requirements are satisfied by the model's output.
3. If any requirements fail, try to repair.

In [ ]:
# Import necessary libraries
import mellea
from mellea.stdlib.requirement import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

# Display utilities
from IPython.display import display, Markdown

# Format code cells with black
import jupyter_black
jupyter_black.load() 

In [ ]:
# Add a requirements list that can be used for validation and repair exercises
requirements = [
    req("The email should have a salutation"),
    req(
        "Use only lower-case letters",
        validation_fn=simple_validate(lambda x: x.lower() == x),
    ),
    check("Do not mention purple elephants."),
]

In [ ]:
def write_email(m: mellea.MelleaSession, name: str, notes: str) -> str:
    """
    Generate a short email using the provided name and notes.

    This function instructs the Mellea session `m` to write an email using the
    global `requirements` list and a rejection-sampling strategy to try multiple
    candidates (loop_budget=5). If validation passes, the validated result is
    returned. If validation fails after sampling, the first sampled generation
    is returned as a fallback.

    Args:
        m: An active MelleaSession used to run the instruction.
        name: Recipient name to be interpolated into the prompt.
        notes: Notes to include in the body of the email.

    Returns:
        A string containing the generated email text.
    """
    # Ask the model to write an email, passing validation requirements and
    # a sampling strategy that will retry up to 5 times if candidates fail.
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )

    # If the instruction succeeded and a validated result is available,
    # return the validated result (preferred).
    if email_candidate.success:
        return str(email_candidate.result)

    # Otherwise, fall back to the first sampled generation (best-effort).
    # This ensures the function always returns some text even if validation failed.
    return email_candidate.sample_generations[0].value

In [ ]:
# Create a Mellea model using the granite3.3:8b model and the ollama inference engine
m = mellea.start_session()

# Generate an email using the write_email function
email = write_email(
    m,
    "Olivia",
    """Olivia helped the lab over the last few weeks by organizing intern events, 
       advertising the speaker series, and handling issues with snack delivery.""",
)

# Display the generated email
display(Markdown(email))

# Exercises

1. Can you change the requirements to only return upper case letters
2. What happens if you change the note to say that the purple elephant was the highlight.
3. Change the requirements, checks and note to write a letter of your choice.